### Graph Algorithms
* 图用来表示对象之间的关系
* 图是顶(vertices)和边(edges)的集合.顶表示对象,边表示关系.
* a graph G is simply a set V of vertices and a collection E of pairs of vertices from V , called edges  
如果边的表示元组(u,v)有序,称为有向图;否则,称为无向图.
可以存在相同的边,称为平行边(parallel edges)  
如果边(u,u),称为边自循环(self-loop).  
没有平行边和边自循环的图称为简单图.  
因此简单图的边是顶点对的集合(Set).

路径(path)是前后相继的边的序列.  
循环(cycle)是起点和终点相同的路径.

当有从u到v的边时,称u可达v(reach)
当有从u到v的路径时,称u可连v(connect)  
当一张图上任意两个定点都是可连的,称为连接(connected),如果任意两个节点都是相互可达的,u reaches v and v reaches u,称为强连接(strongly connected).

包含所有边的子图称为支持子图(spanning subgraph)  
最大可连接子图称为连接元素(connected components)
没有循环的子图称为森林(forest)  
树是可连的森林.

图的特性:
1. 每个顶点涉及到的图的总和是边数量的两倍.
2. 在有向图中,每个顶的起点边=每个顶的终点边,并且起点边的总和=终点边的总和=边数
3. m条边,n顶点的图,有向图:m<=n(n-1),无向图:m<=n(n-1)/2
4. 对于可连图 m>=n-1
5. 对于树 m=n-1
6. 对于森林 m<=n-1

#### 图的抽象数据结构
    edge
1. endpoints( ), 返回(u, v),u为起点,v为终点
2. opposite(v), 返回另一个点  

    Graph
1. vertex count( ),返回图的顶点的个数
2. vertices( ) ,返回全部顶点的迭代
3. edge count( ),返回边的个数
4. edges( ),返回全部边的迭代
5. get edge(u,v),存在则返回u到v的边,否则返回None
6. degree(v, out=True),对于无向图,返回涉及顶点v的全部边的数量
7. incident edges(v, out=True),对于无向图,返回涉及顶点v的全部边的迭代
8. insert vertex(x=None),创造包含元素x的顶点
9. insert edge(u, v, x=None),创造从u到v,包含元素x的边
10. remove vertex(v),去除顶点已经全部和它相关的边
11. remove edge(e)

#### 图的实现结构
1. edgelist 维持一个无序的list存储全部的边 --难以有效定位边
2. adjacency list 对每一个顶点,维持一个包含指向其他顶点的边的list
3. adjacency map 对每一个顶点,维持一个包含指向其他顶点的边的map,以其他顶点为key,边位value
4. adjacency matrix 对于n个顶点,维持n\*n的矩阵,每个entry存一个边.

![Screenshot from 2017-04-06 11-49-03.png](https://ooo.0o0.ooo/2017/04/06/58e5bacd24892.png)

##### adjacency list (邻接表)
![Screenshot from 2017-04-06 12-45-26.png](https://ooo.0o0.ooo/2017/04/06/58e5c7f98e1aa.png)

##### adjacency map (邻接字典)
![Screenshot from 2017-04-06 12-55-09.png](https://ooo.0o0.ooo/2017/04/06/58e5ca403aa55.png)
改善了邻接表,get_edge变成了$O(1)$

##### adjacency matrix (邻接矩阵)
![Screenshot from 2017-04-06 12-59-53.png](https://ooo.0o0.ooo/2017/04/06/58e5cb5c5579c.png)
最大优势是 edge (u, v)变成$O(1)$,但空间占用从其他方法的m+n,变成了n\*n.

#### 图的实现
使用邻接字典

In [49]:
class Vertex:
    __slots__="_element"
    
    def __init__(self,x):
        self._element=x
        
    def element(self):
        return self._element
    
    def __hash__(self):
        return hash(id(self))
    
    def __str__(self):
        return "Vertex:{}".format(self._element)
    def __repr__(self):
        return str(self)
    def __lt__(self,other):
        return self._element<other._element
class Edge:
    __slots__="_origin","_destination","_element"
    
    def __init__(self,u,v,x):
        self._origin=u
        self._destination=v
        self._element=x
        
    def endpoints(self):
        return (self._origin,self._destination)
    
    def opposite(self,v):
        return self._destination if v is self._origin else self._origin
    def __hash__(self):
        return hash((self._origin,self._destination))
    def __str__(self):
        return "Edge:{}".format(self._element)
    def __repr__(self):
        return "Edge:{}".format(self._element)
        
    

class Graph:
    def __init__(self,directed=False):
        #装顶点的字典
        self._outgoing={}
        
        self._incoming={} if directed else self._outgoing
    
    def is_directed(self):
        return self._incoming is not self._outgoing
    
    def vertex_count(self):
        #返回顶点数
        return len(self._outgoing)
    
    def vertices(self):
        #返回顶点的迭代
        return self._outgoing.keys()
    
    def edge_count(self):
        #返回边数
        total=sum(len(self._outgoing[v]) for v in self._outgoing)
        
    def edges(self):
        #返回边的迭代,是一个set
        result=set()
        for secondary_map in self._outgoing.values():
            result.update(secondary_map.values())
        return result
    
    def get_edge(self,u,v):
        #返回源头为u,目的为v的边
        return self._outgoing[u].get(v)
    
    def degree(self,v,outgoing=True):
        #返回以v为源头的边的数量
        maps=self._outgoing if outgoing else self._incoming
        return len(maps[v])
    
    def incident_edges(self, v, outgoing=True):
        #返回以v为源头的边的迭代
        maps=self._outgoing if outgoing else self._incoming
        for edge in maps[v].values():
            yield edge
            
    def insert_vertex(self,x=None):
        #插入一个空字典
        v=Vertex(x)
        self._outgoing[v]={}
        if self.is_directed():
            self._incoming[v]={}
        return v
        
    def insert_edge(self,u,v,x=None):
        e=Edge(u,v,x)
        self._outgoing[u][v]=e
        self._incoming[v][u]=e
    
g=Graph()
a=g.insert_vertex("a")
b=g.insert_vertex("b")
c=g.insert_vertex("c")
d=g.insert_vertex("d")
e=g.insert_vertex("e")
g.insert_edge(a,b,"ab")
g.insert_edge(b,c,"bc")
g.insert_edge(b,d,"bd")
print(g._outgoing)

{Vertex:a: {Vertex:b: Edge:ab}, Vertex:b: {Vertex:a: Edge:ab, Vertex:c: Edge:bc, Vertex:d: Edge:bd}, Vertex:c: {Vertex:b: Edge:bc}, Vertex:e: {}, Vertex:d: {Vertex:b: Edge:bd}}


#### 图的遍历
如果一个遍历能以线性时间访问全部的顶点和边,那么这个遍历就是有效的.  
图遍历的关键问题是回答可达性.
深度优先和广度优先是两种常用算法.
##### 深度优先搜索 depth-first search (DFS)

伪代码:
```
Algorithm DFS(G,u):
{We assume u has already been marked as visited}
Input: A graph G and a vertex u of G
Output: A collection of vertices reachable from u, with their discovery edges
for each outgoing edge e = (u, v) of u do 对每个顶点,找到所有可达的顶点
if vertex v has not been visited then     没有访问过则标记
Mark vertex v as visited (via edge e).
Recursively call DFS(G,v).                递归访问没有标记的顶点
```

![Screenshot from 2017-04-06 14-20-03.png](https://ooo.0o0.ooo/2017/04/06/58e5de2b20ce3.png)
1. 深度优先搜索把图转化成了树
2. 深度优先搜索找出了所有与顶点s可达的顶点.

##### 深度优先搜索的实现
我们使用并查集来标记找到的顶点,并查集是一个存储父节点的字典.

In [29]:
def DFS(g, u, discovered):
    for edge in g.incident_edges(u):
        v=edge.opposite(u)
        if v not in discovered:
            #父顶点为子顶点的值
            discovered[v]=u
            DFS(g,v,discovered)
result={a:None}
DFS(g,a,result)
print(result)

{Vertex:a: None, Vertex:b: Vertex:a, Vertex:c: Vertex:b, Vertex:d: Vertex:b}


通过取得的并查集,我们重组路径

In [30]:
def construct_path(u, v, discovered):
    path=[]
    if v not in discovered:
        raise KeyError("no such key")
    else:
        while v is not u:
            path.append(v)
            v=discovered[v]
        path.append(v)
        path.reverse()
        return path
    
print(construct_path(a,c,result))

[Vertex:a, Vertex:b, Vertex:c]


图中的每个顶点不一定都是可达的,也就是说一个图可能包括多个树  
我们把包含图中所有树的集合,称为森林(forest)

In [32]:
from itertools import chain
def DFS_all(g):
    forest=[]
    for u in g.vertices():
        if u not in chain(*(tree.keys() for tree in forest)):
            newtree={u:None}
            forest.append(newtree)
            DFS(g,u,newtree)
    return forest

DFS_all(g)#森林里有两棵树

[{Vertex:a: None, Vertex:b: Vertex:a, Vertex:c: Vertex:b, Vertex:d: Vertex:b},
 {Vertex:e: None}]

In [41]:
def listflat(l):
    result=[]
    for item in l:
        if isinstance(item,list):
            result+=listflat(item)
        else:
            result.append(item)
    return result

def listflat2(l):
     return list(i for li in ([item]  if not isinstance(item,list) else listflat2(item) for item in l) for i in li)
l=[[1,2,3],[4],5]
print(listflat(l))
print(listflat2(l))

[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]


#### 广度优先算法 breadth-first search (BFS)
引入了一个层次的概念,分层进行搜索
![Screenshot from 2017-04-06 16-27-19.png](https://ooo.0o0.ooo/2017/04/06/58e5fc22c6804.png)
原理:
1. 将顶点入队列
2. 当队列为非空时，继续执行，否则算法结束
3. 出队列取得队列头顶点V；访问并标记为已访问
4. 查找顶点V的第一个邻接顶点W
5. 若V的邻接顶点W未被访问过的，则W入队列
6. 继续查找顶点V的另一个新的邻接顶点W，转到步骤2

In [42]:
#广度优先算法没有使用递归,因为它按层次顺序进行处理.
def BFS(g,s,discovered):
    level=[s]
    while len(level)>0:
        next_level=[]
        for u in level:
            for edge in g.incident_edges(u):
                v=edge.opposite(u)
                if v not in discovered:
                    discovered[v]=u
                    next_level.append(v)
        level=next_level
result={a:None}
BFS(g,a,result)
print(result)

{Vertex:a: None, Vertex:b: Vertex:a, Vertex:c: Vertex:b, Vertex:d: Vertex:b}


#### 最短路径
对于有权图(Weighted Graphs),计算其最短路径是非常常见的需求,比如规划城市间的交通路线,规划快递站等.
对于每条边的权重都一样的情况,对于顶点(u,v),u的广度优先搜索直接返回了到v的最短路径.
##### 有权图(Weighted Graphs)
每条边都有一个权重
![Screenshot from 2017-04-06 17-00-08.png](https://ooo.0o0.ooo/2017/04/06/58e603a9696dc.png)
对于有权图来说,一条路径的权重,等于该路径上全部边的权重和
我们把d(u,v)称为u到v的权重最短路径
![Screenshot from 2017-04-06 17-05-52.png](https://ooo.0o0.ooo/2017/04/06/58e6050165131.png)

##### Dijkstra’s Algorithm计算最短路径
Dijkstra(迪杰斯特拉)算法是典型的单源最短路径算法，用于计算一个节点到其他所有节点的最短路径。主要特点是以起始点为中心向外层层扩展，直到扩展到终点为止

算法思想：设G=(V,E)是一个带权有向图，把图中顶点集合V分成两组，第一组为已求出最短路径的顶点集合（用S表示，初始时S中只有一个源点，以后每求得一条最短路径 , 就将加入到集合S中，直到全部顶点都加入到S中，算法就结束了），第二组为其余未确定最短路径的顶点集合（用U表示），按最短路径长度的递增次序依次把第二组的顶点加入S中。在加入的过程中，总保持从源点v到S中各顶点的最短路径长度不大于从源点v到U中任何顶点的最短路径长度。此外，每个顶点对应一个距离，S中的顶点的距离就是从v到此顶点的最短路径长度，U中的顶点的距离，是从v到此顶点只包括S中的顶点为中间顶点的当前最短路径长度。
伪代码:
```
Algorithm ShortestPath(G, s):
Input: A weighted graph G with nonnegative edge weights, and a distinguished
vertex s of G.
Output: The length of a shortest path from s to v for each vertex v of G.
Initialize D[s] = 0 and D[v] = ∞ for each vertex v != s.
Let a priority queue Q contain all the vertices of G using the D labels as keys.
while Q is not empty do
{pull a new vertex u into the cloud}
u = value returned by Q.remove min()
for each vertex v adjacent to u such that v is in Q do
{perform the relaxation procedure on edge (u, v)}
if D[u] + w(u, v) < D[v] then
D[v] = D[u] + w(u, v)
Change to D[v] the key of vertex v in Q.
return the label D[v] of each vertex v
```

松弛:代码中D保存各个节点到源点的距离值估计(上界值)，P保存节点的最短路径上的前驱节点，g保存边的权值，其中不存在的边的权值为inf。松弛就是说，假设节点 u 和节点 v 事先都有一个最短距离的估计(例如测试代码中的7和13)，如果现在要松弛边(u,v)，也就是对从节点 u 通过边(u,v)到达节点 v，将这条路径得到节点 v 的距离估计值(7+3=10)和原来的节点 v 的距离估计值(13)进行比较，如果前者更小的话，就表示我们可以放弃在这之前确定的从源点到节点 v 的最短路径，改成从源点到节点 u，然后节点 u 再到节点 v，这条路线距离会更短些，这也就是发生了一次松弛！(测试代码中10<13，所以要进行松弛，此时D[v]变成10，而它的前驱节点也变成了 u)

In [44]:
#边松弛
def relax(g,s,v,d,p):
    weightsv=g.get_edge(s,v)._element
    if d[s]+weightsv<d[v]:
        d[v]=d[s]+weightsv
        p[v]=s
        return True
    return False

显然，如果你随机地对边进行松弛，那么与该边有关的节点的距离估计值就会慢慢地变得更加准确，这样的改进会在整个图中进行传播，如果一直这么松弛下去的话，最终整个图所有节点的距离值都不会发生变化的时候我们就得到了从源点到所有节点的最短路径值。

每次松弛可以看作是向最终解前进了“一步”，我们的目标自然是希望松弛的次数越少越好，关键就是要确定松弛的次数和松弛的顺序(好的松弛顺序可以让我们直接朝着最优解前进，缩短算法运行时间)，后面要介绍的图中的Bellman-Ford算法、Dijkstra算法以及DAG上的最短路径问题都是如此。

让我们从递归的角度来看这种算法:
这个算法的核心在于维持以顶点到源顶点的距离为键一个最小堆.
1. 最初,最小为源到源自身的距离,源出堆,源的邻居入堆,更新最小值
2. 除了最小值路径,源想要到底最小值路径的顶点,只能通过该顶点的邻居或者最小值中其他顶点,因为权重大于零,其他路径必大于最小值路径,因此每一次出堆都能确定到一个顶点的最小路径.

In [52]:
from heapq import heappush, heappop
def shortestpath(g,s):
    #最小堆
    d=[]
    #保存顶点到源的距离
    tree={}
    #结果的path
    result={s:None}
    #已经出堆的顶点
    hasout=[]
    #遍历全部节点,包括不可达的顶点
    for v in g.vertices():
        if v==s:
            heappush(d,(0,v))
            tree[v]=0
        else:
            heappush(d,(float("inf"),v))
            tree[v]=float("inf")
    while d:
        stov,v=heappop(d)
        hasout.append(v)
        for edge in g.incident_edges(v):
            nv=edge.opposite(v)
            if nv not in hasout:
                relax(g,v,nv,tree,result)
                heappush(d,(tree[nv],nv))
    return result
mygraph=Graph()
a=mygraph.insert_vertex("a")
b=mygraph.insert_vertex("b")
c=mygraph.insert_vertex("c")
d=mygraph.insert_vertex("d")
e=mygraph.insert_vertex("e")
mygraph.insert_edge(a,b,100)
mygraph.insert_edge(a,c,200)
mygraph.insert_edge(b,d,105)
mygraph.insert_edge(b,e,600)
mygraph.insert_edge(c,e,100)
print(shortestpath(mygraph,a))

{Vertex:a: None, Vertex:b: Vertex:a, Vertex:c: Vertex:a, Vertex:e: Vertex:c, Vertex:d: Vertex:b}


##### 最小生成树 普里姆算法（Prim算法)
图论中的一种算法，可在加权连通图里搜索最小生成树。意即由此算法搜索到的边子集所构成的树中，不但包括了连通图里的所有顶点，且其所有边的权值之和亦为最小。  
普里姆算法（Prim算法)和Dijkstra’s algorithm类似,我们随机选择s,定义一个最小堆,以边的权重为key,每次从堆中取出与当前顶点相连且权重最小的节点,把它加入结果集,对其后继节点松弛后入堆,或者直接插入新权重小的后继节点.
![](https://hujiaweibujidao.github.io/images/algos/prim.png)

In [54]:
def prim(g,s):
    result,d={},[(0,None,s)]
    while d:
        _,parent,u=heappop(d)
        if u in result:
            continue
        result[u]=parent
        for edge in g.incident_edges(u):
            heappush(d,(edge._element,u,edge.opposite(u)))
    return result

print(prim(mygraph,a))
    

{Vertex:a: None, Vertex:b: Vertex:a, Vertex:c: Vertex:a, Vertex:e: Vertex:c, Vertex:d: Vertex:b}
